In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

In [2]:
import datetime as dt

In [3]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, extract

In [4]:
# Access the SQLite database
engine = create_engine("sqlite:///hawaii.sqlite")

In [5]:
# Reflect the database into classes
Base = automap_base()
Base.prepare(engine, reflect=True)

In [6]:
# Save references to each table by creating a variable for each of the classes
Measurement = Base.classes.measurement
Station = Base.classes.station

In [7]:
# Create a session link from Python to the database
session = Session(engine)

In [8]:
june_temps = session.query(Measurement).filter(extract('month', Measurement.date) == 6)
june_temps_list = [temp.tobs for temp in june_temps]
print(np.mean(june_temps_list))

74.94411764705882


In [9]:
june_df = pd.DataFrame(june_temps_list, columns=["temps"])
june_df.describe()

,temps
count,1700.000000
mean,74.944118
std,3.257417
min,64.000000
25%,73.000000
50%,75.000000
75%,77.000000
max,85.000000


In [10]:
dec_temps = session.query(Measurement).filter(extract('month', Measurement.date) == 12)
dec_temps_list = [temp.tobs for temp in dec_temps]
print(np.mean(dec_temps_list))

71.04152933421226


In [11]:
dec_df = pd.DataFrame(dec_temps_list, columns=["temps"])
dec_df.describe()

,temps
count,1517.000000
mean,71.041529
std,3.745920
min,56.000000
25%,69.000000
50%,71.000000
75%,74.000000
max,83.000000


# June and December weather analysis

### Key statistical data for temperature in June (2010-2017) across all of the stations and years
count: 1700<br>
mean: 74.94<br>
std: 3.26<br>
min: 64.00<br>
25%: 73.00<br>
50%: 75.00<br>
75%: 77.00<br>
max: 85.00<br>


### Key statistical data for temperature in December across all of the stations and years
count: 1517<br>
mean: 71.04<br>
std: 3.75<br>
min: 56.00<br>
25%: 69.00<br>
50%: 71.00<br>
75%: 74.00<br>
max: 83.00<br>

### Temperature Summary
The average June temperature of 74.94 is higher than the average December temperature of 71.04. This is reflected in the other statiscal data where June has higher min, max, and percentiles compared to December. However, the differences are not significant and we can assume from this data that the temperatures are comparable for both months.
    
### Recommendations for future analysis
Our weather analysis so far has been helpful to understand the temperature trends for June and December. For completeness, we may want to dig into the data further to understand the differences in count. Depending on the months of the year that the shop will be open, we can expand the dataset to provide similar weather analysis. Once we have this complete view of the monthly weather patterns, the next step will be to identify a location in Oahu. The dataset already includes the latitude and longitude of the station which provides a general location area to explore. Additional variables to consider is the proximity to beaches, surfing conditions, and availability of other similar shops.